<a href="https://colab.research.google.com/github/eduzegarra/grade_01/blob/main/proyecto_papa_v02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Proyecto Papa Sostenible: Integración, Limpieza y Análisis Exploratorio**


Librerías externas necesarias

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## **1. Lectura, limpieza e integración de base de datos**

### 1.1. Lectura de base de datos original en Excel

Leemos primer libro del archivo Excel (volumen) asignado a objeto "vol" (en este caso subimos primero el archivo al entorno para leerlo)

In [ ]:
vol = pd.read_excel('/content/sample_data/base_completa_1997_2021.xls', sheet_name='volumen')
vol

In [11]:
vol.rename(columns={'m_1':'m_01','m_2':'m_02','m_3':'m_03','m_4':'m_04','m_5':'m_05','m_6':'m_06',
                    'm_7':'m_07','m_8':'m_08', 'm_9':'m_09'}, inplace=True)

In [ ]:
vol.isna().sum().sum()

Leemos segundo libro (precios) asignado a objeto "pre"

In [ ]:
pre = pd.read_excel('/content/sample_data/base_completa_1997_2021.xls', sheet_name='precio')
pre.head()

In [14]:
pre.rename(columns={'m_1':'m_01','m_2':'m_02','m_3':'m_03','m_4':'m_04','m_5':'m_05','m_6':'m_06',
                    'm_7':'m_07','m_8':'m_08', 'm_9':'m_09'}, inplace=True)

In [ ]:
pre.isna().sum().sum()

La columna 'Variable' identifica la provincia, cambiamos el nombre de la columna a 'provincia'

In [16]:
vol.rename(columns={'Variable':'provincia'}, inplace=True)

Las columnas m_01 a m_12 denotan meses. Necesitamos convertir al mes como una variable, para tener en cada fila una observacion, pasar base en formato ancho a largo (wide to long) usando el método **melt**

In [53]:
vol_long=pd.melt(vol,id_vars=['producto','year','provincia'], 
                var_name='mes', value_name='volumen')

Lo mismo con la base de precios

In [43]:
pre_long=pd.melt(pre,id_vars=['producto','year'], 
                var_name='mes', value_name='precio')

### 1.2. Limpieza datos originales

Damos una mirada a cada una de estas bases de datos

primero el asunto de los valores "missing" (sin información)

In [ ]:
vol_long.isna().groupby(vol_long.year)['volumen'].sum().plot()

In [ ]:
pre_long.isna().groupby(pre_long.year)['precio'].sum().plot()
## 963 valores missing

Eliminamos observaciones con valores missing (NaN) en ambas bases

In [70]:
pre_long.dropna(inplace=True)
vol_long.dropna(inplace=True)

In [ ]:
vol_long.isna().sum()

In [73]:
pre_long.isna().sum()

producto    0
year        0
mes         0
precio      0
dtype: int64

### 1.3. Evolución de variables y número de observaciones

In [ ]:
vol_long.groupby(vol_long.year).agg({'provincia':'nunique', 'producto':'nunique', 'mes':['nunique','count']})

In [ ]:
plt.figure(figsize=(9,5))
vol_long.groupby(vol_long.year)['volumen'].sum().plot()
plt.xticks(np.arange(1997,2022,1),rotation=45, fontsize='small')
plt.yticks(np.arange(50000,800000,50000), fontsize='small')
plt.title('Volumen transado por año (t)', fontsize='xx-large')
plt.xlabel('')
plt.show()

In [ ]:
plt.figure(figsize=(9,5))
vol_long.groupby(vol_long.year)['volumen'].count().plot()
plt.xticks(np.arange(1997,2022,1),rotation=45, fontsize='small')
plt.yticks(np.arange(0,1600,200))
plt.title('Número de observaciones por año de volumen', fontsize='xx-large')
plt.xlabel('')
plt.show()

In [ ]:
pre_long.groupby(pre_long.year).agg({'producto':'nunique', 'mes':['nunique','count']})

In [ ]:
plt.figure(figsize=(9,5))
pre_long[pre_long['year']<=2020].groupby(pre_long.year)['precio'].count().plot()
rr=np.arange(1997,2022,1)
plt.xticks(rr,rotation=45, fontsize='small')
plt.yticks(np.arange(20,150,20))
plt.title('Número de observaciones por año en base precios', fontsize='xx-large')
plt.show()

### 1.4. Merge de archivos

Bases 'vol_long' y 'pre_long' tienen una estructura similar y variables en comun y pueden ser integradas (usando método **merge**, con parámetro 'inner')

In [144]:
df_base=vol_long.merge(pre_long, on=['producto','year','mes'], how='inner')

In [ ]:
df_base.shape

In [ ]:
df_base.columns

In [ ]:
df_base.describe(include='all')

In [ ]:
df_base

En objeto generado como **df_base** en cada fila tenemos una observación que identifica la variedad, el año, la provincia, el mes así como el volumen y precio

Notar que la variable precio corresponde a la variedad y mes, pero no varía por provincia de origen (el precio se forma en MML)

Creamos variable 'ym' que identifica año y mes por observacion

In [150]:
df_base['ym']=df_base['year'].astype(str)+df_base['mes']

In [ ]:
df_base.sort_values('ym', ascending=True)

La base df_base ha pasado por un primer proceso de limpieza e integración: 
    
    (i) convertimos bases en formato ancho a largo
    
    (ii) eliminamos valores NaN en bases de volumen y precios
    
    (iii) juntamos bases de volumen y precios
    
    (iv) cambiamos algunos nombres de variables

Esta es una primera base "limpia" y "ordenada" con la que se puede iniciar el análisis exploratorio.

Tiene 21,333 observaciones con valores para precios y volúmenes de 11 variedades de papa

## **2. Primer análisis exploratorio**

## 2.1. Mirada general


Veremos el comportamiento de las variedades (su dinámica), primero cambiamos el nombre de la columna 'producto' por 'variedad'

Renombramos variable 'producto' por 'variedad'

In [152]:
df_base.rename(columns={'producto':'variedad'}, inplace=True)

Miramos las variables de volumen, precio, años y provincias de origen por variedad usando **groupby**

In [ ]:
df_base.groupby(df_base['variedad']).agg({'volumen':['count','mean', 'min','max'],'precio':['mean','min','max'], 'year':['min','max'], 'provincia':'nunique'})

In [ ]:
df_base.groupby(df_base['variedad'])['volumen'].mean().plot.bar()

In [ ]:
df_base.groupby(df_base['variedad'])['precio'].mean().plot.bar()

In [ ]:
df_base.groupby(df_base['variedad']).agg({'provincia':'nunique'}).plot.bar()

## 2.2.  Dinámica de oferta y precios por variedad

### Papa Blanca

In [ ]:
plt.figure(figsize=(8,5))
vol_blanca=df_base[['year','volumen']][df_base.variedad=="Papa Blanca"].groupby('year').sum()
plt.bar(vol_blanca.index,vol_blanca.volumen)
plt.title("Papa Blanca: Volumen", fontsize='xx-large')
plt.xticks(vol_blanca.index,rotation=45, fontsize='small')
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
pre_blanca=df_base[['year','precio']][df_base.variedad=="Papa Blanca"].groupby('year').mean()
plt.plot(pre_blanca.index,pre_blanca.precio)
plt.title("Papa Blanca: Precio", fontsize='xx-large')
plt.xticks(pre_blanca.index,rotation=45, fontsize='small')
plt.show()

### Papa Yungay

In [ ]:
plt.figure(figsize=(8,5))
vol_yungay=df_base[['year','volumen']][df_base.variedad=="Papa Yungay"].groupby('year').sum()
plt.bar(vol_yungay.index,vol_yungay.volumen)
plt.title("Papa Yungay: Volumen", fontsize='xx-large')
plt.xticks(vol_blanca.index,rotation=45, fontsize='small')
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
pre_yungay=df_base[['year','precio']][df_base.variedad=="Papa Yungay"].groupby('year').mean()
plt.plot(pre_yungay.index,pre_yungay.precio)
plt.title("Papa Yungay: Precio", fontsize='xx-large')
plt.xticks(pre_blanca.index,rotation=45, fontsize='small')
plt.yticks(np.arange(0.1,1.7,.1))
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
pre_yungay=df_base[['year','precio']][df_base.variedad=="Papa Yungay"].groupby('year').mean()
plt.plot(pre_yungay.index,pre_yungay.precio)
plt.plot(pre_blanca.index,pre_blanca.precio)
plt.title("Papa Blanca y Yungay: Precio", fontsize='xx-large')
plt.xticks(pre_blanca.index,rotation=45, fontsize='small')
plt.yticks(np.arange(0.1,1.7,.1))
plt.show()

### Papa Huayro

In [ ]:
plt.figure(figsize=(8,5))
vol_huayro=df_base[['year','volumen']][df_base.variedad=="Papa Huayro"].groupby('year').sum()
plt.bar(vol_huayro.index,vol_huayro.volumen)
plt.title("Papa Huayro: Volumen")
plt.xticks(vol_huayro.index,rotation=45, fontsize='small')
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
pre_huayro=df_base[['year','precio']][df_base.variedad=="Papa Huayro"].groupby('year').mean()
plt.plot(pre_huayro.index,pre_huayro.precio)
plt.title("Papa Huayro: Precio", fontsize='xx-large')
plt.xticks(pre_huayro.index,rotation=45, fontsize='small')
plt.yticks(np.arange(0.1,1.7,.2))
plt.show()

### Papa Canchan

In [ ]:
plt.figure(figsize=(8,5))
vol_canchan=df_base[['year','volumen']][df_base.variedad=="Papa Canchan"].groupby('year').sum()
plt.bar(vol_canchan.index,vol_canchan.volumen)
plt.title("Papa Canchan: Volumen", fontsize='xx-large')
plt.xticks(vol_blanca.index,rotation=45, fontsize='small')
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
pre_canchan=df_base[['year','precio']][df_base.variedad=="Papa Canchan"].groupby('year').mean()
plt.plot(pre_canchan.index,pre_canchan.precio)
plt.title("Papa Canchan: Precio", fontsize='xx-large')
plt.xticks(pre_blanca.index,rotation=45, fontsize='small')
plt.yticks(np.arange(.1,1.7,.2))
plt.show()

### Papa Amarilla

In [ ]:
plt.figure(figsize=(8,5))
vol_amarilla=df_base[['year','volumen']][df_base.variedad=="Papa Amarilla"].groupby('year').sum()
plt.bar(vol_amarilla.index,vol_amarilla.volumen)
plt.title("Papa Amarilla: Volumen", fontsize='xx-large')
plt.xticks(vol_amarilla.index,rotation=45, fontsize='small')
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
pre_amarilla=df_base[['year','precio']][df_base.variedad=="Papa Amarilla"].groupby('year').mean()
plt.plot(pre_amarilla.index,pre_amarilla.precio)
plt.title("Papa Amarilla: Precio", fontsize='xx-large')
plt.xticks(pre_amarilla.index,rotation=45, fontsize='small')
plt.yticks(np.arange(.1,2.4,.2))
plt.show()

### Papa Color

In [ ]:
plt.figure(figsize=(8,5))
vol_color=df_base[['year','volumen']][df_base.variedad=="Papa Color"].groupby('year').sum()
plt.bar(vol_color.index,vol_color.volumen)
plt.title("Papa Color: Volumen", fontsize='xx-large')
plt.xticks(vol_color.index,rotation=45, fontsize='small')
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
pre_color=df_base[['year','precio']][df_base.variedad=="Papa Color"].groupby('year').mean()
plt.plot(pre_color.index,pre_color.precio)
plt.title("Papa Color: Precio", fontsize='xx-large')
plt.xticks(pre_color.index,rotation=45, fontsize='small')
plt.yticks(np.arange(.1,1.6,.2))
plt.show()

### Papa Unica

In [ ]:
plt.figure(figsize=(8,5))
vol_unica=df_base[['year','volumen']][df_base.variedad=="Papa Unica"].groupby('year').sum()
plt.bar(vol_unica.index,vol_unica.volumen)
plt.title("Papa Unica: Volumen", fontsize='xx-large')
plt.xticks(vol_color.index,rotation=45, fontsize='small')
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
pre_unica=df_base[['year','precio']][df_base.variedad=="Papa Unica"].groupby('year').mean()
plt.plot(pre_unica.index,pre_unica.precio)
plt.title("Papa Unica: Precio", fontsize='xx-large')
plt.xticks(pre_color.index,rotation=45, fontsize='small')
plt.yticks(np.arange(.1,1.6,.2))
plt.show()

### Papa Peruanita

In [ ]:
plt.figure(figsize=(8,5))
vol_peruanita=df_base[['year','volumen']][df_base.variedad=="Papa Peruanita"].groupby('year').sum()
plt.bar(vol_peruanita.index,vol_peruanita.volumen)
plt.title("Papa Peruanita: Volumen", fontsize='xx-large')
plt.xticks(vol_color.index,rotation=45, fontsize='small')
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
pre_peruanita=df_base[['year','precio']][df_base.variedad=="Papa Peruanita"].groupby('year').mean()
plt.plot(pre_peruanita.index,pre_peruanita.precio)
plt.title("Papa Peruanita: Precio", fontsize='xx-large')
plt.xticks(pre_color.index,rotation=45, fontsize='small')
plt.yticks(np.arange(.1,2.4,.2))
plt.show()

### Papa Perricholi

In [ ]:
plt.figure(figsize=(8,5))
vol_perricholi=df_base[['year','volumen']][df_base.variedad=="Papa Perricholi"].groupby('year').sum()
plt.bar(vol_perricholi.index,vol_perricholi.volumen)
plt.title("Papa Perricholi: Volumen", fontsize='xx-large')
plt.xticks(vol_color.index,rotation=45, fontsize='small')
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
pre_perricholi=df_base[['year','precio']][df_base.variedad=="Papa Perricholi"].groupby('year').mean()
plt.plot(pre_perricholi.index,pre_perricholi.precio)
plt.title("Papa Perricholi: Precio", fontsize='xx-large')
plt.xticks(pre_color.index,rotation=45, fontsize='small')
plt.yticks(np.arange(.1,1.6,.2))
plt.show()

### Papa Huamantanga

In [ ]:
plt.figure(figsize=(8,5))
vol_huamantanga=df_base[['year','volumen']][df_base.variedad=="Papa Huamantanga"].groupby('year').sum()
plt.bar(vol_huamantanga.index,vol_huamantanga.volumen)
plt.title("Papa Huamantanga: Volumen", fontsize='xx-large')
plt.xticks(vol_color.index,rotation=45, fontsize='small')
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
pre_huamantanga=df_base[['year','precio']][df_base.variedad=="Papa Huamantanga"].groupby('year').mean()
plt.plot(pre_huamantanga.index,pre_huamantanga.precio)
plt.title("Papa Huamantanga: Precio", fontsize='xx-large')
plt.xticks(pre_color.index,rotation=45, fontsize='small')
plt.yticks(np.arange(.1,2.4,.2))
plt.show()

### Papa Negra Andina

In [ ]:
plt.figure(figsize=(8,5))
vol_negra=df_base[['year','volumen']][df_base.variedad=="Papa Negra Andina"].groupby('year').sum()
plt.bar(vol_negra.index,vol_negra.volumen)
plt.title("Papa Negra Andina: Volumen", fontsize='xx-large')
plt.xticks(vol_color.index,rotation=45, fontsize='small')
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
pre_negra=df_base[['year','precio']][df_base.variedad=="Papa Negra Andina"].groupby('year').mean()
plt.plot(pre_negra.index,pre_negra.precio)
plt.title("Papa Negra Andina: Precio", fontsize='xx-large')
plt.xticks(pre_color.index,rotation=45, fontsize='small')
plt.yticks(np.arange(.1,1.8,.2))
plt.show()

### Papas con precios similares a Papa Blanca

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(pre_yungay.index,pre_yungay.precio,label='Yungay')
plt.plot(pre_blanca.index,pre_blanca.precio, label='Blanca')
plt.plot(pre_canchan.index,pre_canchan.precio, label='Canchan')
plt.plot(pre_color.index,pre_color.precio, label='Color')
plt.plot(pre_perricholi.index,pre_perricholi.precio, label='Perricholi')
plt.title("Papa Blanca, Yungay, Canchan, Color, Perricholi: Precio", fontsize='large')
plt.xticks(pre_blanca.index,rotation=45, fontsize='small')
plt.legend()
plt.show()

### Papas con precios similares a papa amarilla

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(pre_amarilla.index,pre_amarilla.precio, label='Amarilla')
plt.plot(pre_peruanita.index,pre_peruanita.precio, label='Peruanita')
plt.plot(pre_huamantanga.index,pre_huamantanga.precio, label='Huamantanga')
plt.title("Papa Amarilla, Peruanita, Huamantanga", fontsize='large')
plt.xticks(pre_blanca.index,rotation=45, fontsize='small')
plt.legend()
plt.show()

### Precios de huayro y negra andina

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(pre_huayro.index,pre_huayro.precio, label='Huayro')
plt.plot(pre_negra.index,pre_negra.precio, label='Negra Andina')
plt.title("Papa Huayro, Negra Andina", fontsize='large')
plt.xticks(pre_blanca.index,rotation=45, fontsize='small')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(pre_blanca.index,pre_blanca.precio, label='Blanca', color='gray')
plt.plot(pre_amarilla.index,pre_amarilla.precio, label='Amarilla')
plt.plot(pre_huayro.index,pre_huayro.precio, label='Huayro')
plt.title("Papa Blanca, Amarilla y Huayro", fontsize='x-large')
plt.xticks(pre_blanca.index,rotation=45, fontsize='small')
plt.legend()
plt.show()

## 2.3. Análisis de provincias de origen

### Papa Blanca

In [ ]:
prv_blanca=df_base[['provincia','volumen']][df_base.variedad=="Papa Blanca"].groupby('provincia').sum()
prv_blanca.sort_values(by='volumen', ascending=False).head(20)

### Papa Amarilla

In [ ]:
prv_amarilla=df_base[['provincia','volumen']][df_base.variedad=="Papa Amarilla"].groupby('provincia').sum()
prv_amarilla.sort_values(by='volumen', ascending=False).head(20)

### Papa Huayro

In [ ]:
prv_huayro=df_base[['provincia','volumen']][df_base.variedad=="Papa Huayro"].groupby('provincia').sum()
prv_huayro.sort_values(by='volumen', ascending=False).head(20)

In [ ]:
prv_yungay=df_base[['provincia','volumen']][df_base.variedad=="Papa Yungay"].groupby('provincia').sum()
prv_yungay.sort_values(by='volumen', ascending=False).head(20)

## 2.4. Peso relativo de las provincias

Creamos variable de valor (precio*volumen) para cada observación. Esta variable está en millones de soles

In [231]:
df_base['valor']=(df_base['volumen']*df_base['precio'])/1000

In [ ]:
df_base.valor.describe()

In [235]:
vv_valor=pd.DataFrame(df_base.groupby('year')['valor'].sum())


In [ ]:
plt.figure(figsize=(8,5))
plt.bar(vv_valor.index,vv_valor.valor)
plt.suptitle("Valor de transacciones de Papa 1997-2020",fontsize='x-large')
plt.title("Millones de soles")
plt.xticks(vv_valor.index,rotation=45, fontsize='small')
plt.show()

Generamos ranking de provincias de acuerdo a valor total de papa enviada a Lima entre 1997 y mayo 2021

In [237]:
rank_prov=df_base.groupby(df_base.provincia)['valor'].sum()

In [238]:
prov_19=rank_prov.sort_values(ascending=False)[0:19].sum()

In [239]:
prov_resto=rank_prov.sort_values(ascending=False)[19:121].sum()

In [ ]:
prov_19/(prov_19+prov_resto)


19 provincias generan el 96.4% del valor total de la papa enviada a Lima, entonces tiene sentido acotar la base solamente a esas 20 provincias

In [245]:
r19=rank_prov.sort_values(ascending=False)[0:19]

In [ ]:
r19.index

In [255]:
df_base2=df_base[df_base["provincia"].isin(r19.index)]

In [ ]:
df_base2.provincia.value_counts()

In [ ]:
df_base2.describe(include='all')

## 2.4. Análisis por región natural de origen

In [ ]:
df_base2['costa']=df_base2['provincia'].isin(['Huaral','Lima','Barranca',
                'Nazca','Canete','Huaura','Arequipa','Ica']).astype('int')

Peso de cada región en valor de producción por variedad

In [ ]:
v_costa=df_base2[df_base2['costa']==1].groupby('variedad')['valor'].sum()
v_sierra=df_base2[df_base2['costa']==0].groupby('variedad')['valor'].sum()
v_total=v_costa+v_sierra
r_sierra=(v_sierra/v_total)*100
plt.bar(r_sierra.index,r_sierra)
plt.xticks(rotation=90)
plt.title("Porcentaje valor de oferta proveniente de sierra")
plt.show()

In [ ]:
plt.figure(figsize=(9,5))
ll_costa=df_base2[(df_base2.costa==1) & (df_base2.year<=2020)].groupby('year')['valor'].sum()
ll_sierra=df_base2[(df_base2.costa==0) & (df_base2.year<=2020)].groupby('year')['valor'].sum()
plt.plot(ll_costa.index, ll_costa, label='Costa')
plt.plot(ll_sierra.index, ll_sierra, label='Sierra')
plt.legend(fontsize='large')
plt.suptitle('Valor de transacciones 1997-2020 por region natural', fontsize='xx-large')
plt.title('Millones de soles por año')
plt.xticks(ll_costa.index, rotation=45)
plt.show

In [ ]:
plt.figure(figsize=(12,5))
mm_costa=df_base2[df_base2.costa==1].groupby('ym')['valor'].sum()
mm_sierra=df_base2[df_base2.costa==0].groupby('ym')['valor'].sum()
plt.plot(mm_costa.index, mm_costa, label='Costa')
plt.plot(mm_sierra.index, mm_sierra, label='Sierra')
plt.legend(fontsize='large')
plt.suptitle('Valor de transacciones 1997-2020 por region natural', fontsize='xx-large')
plt.title('Millones de soles por mes')
rr=mm_costa.index[np.arange(0,len(mm_costa),11)]
plt.xticks(rr, rotation=90, fontsize='small')
plt.show

La base **df_base2** puede ser utilizada para el modelamiento y análisis del mercado de papa (en sus distintas variedades) de Lima Metropolitana.

In [ ]:
df_base2.describe(include='all')

Tenemos 15,549 observaciones en la base final, 11 variedades, 19 provincias, 291 meses

In [ ]:
df_base2.variedad.value_counts()

In [266]:
df_base2.to_excel('/content/sample_data/df_base2.xlsx')